# Arbeidsløype for grafkonversjon med Grew

1. [konverter ordklasser og morfologiske trekk](#konverter-ordklasser-og-morfologiske-trekk) fra NDT til UD-POS-tags og feats
2. [Legg til "SpaceAfter=No"](#legg-til-spaceafterno) i `MISC`-feltet for tegnsetting som ikke skal ha mellomrom.
3. [Kjør eksisterende regler](#kjør-eksisterende-omskrivingsregler-på-trebanken) på trebanken med et Grew Graph Rewritign System (GRS)
4. [Søk etter setningsmønstre](#steg-1-søk-etter-setningsmønster-i-trebanken)
5. [Skriv en konverteringsregel](#steg-2-skriv-en-konverteringsregel)
6. [Visualiser setningsgrafer](#visualiser-setningsgrafer)

Se [Grewpy-dokumentasjonen](https://grew.fr/python/grewpy.html#grewpy-package) for mer info om APIet i python. 


In [ ]:
# Fix PATH to include OPAM binaries where grewpy_backend is located
import os

os.environ["PATH"] = os.environ["HOME"] + "/.opam/5.2.0/bin:" + os.environ["PATH"]

In [ ]:
import grewpy

grewpy.set_config("ud")  # ud or basic

## Konverter ordklasser og morfologiske trekk

In [ ]:
from pathlib import Path

from grewpy import Corpus

from ndt2ud.morphological_features import convert_morphology
from ndt2ud.parse_conllu import parse_conll_file
from ndt2ud.utils import write_conll

treebank_file = (
    "../spoken_norwegian_resources/treebanks/Norwegian-NynorskLIA/aal_uio_06.conll"
)
UD_treebank_file = "../UD_output.conllu"

# Load the NDT treebank and convert the part of speech and morphological features to UD features
conllu_data = parse_conll_file(Path(treebank_file))
morphdata = convert_morphology(conllu_data)
write_conll(morphdata, UD_treebank_file, drop_comments=False)

original_corpus = Corpus(UD_treebank_file)

## Endre features for token


### Legg til "SpaceAfter=No"

> **OBS** Dette steget har ingen effekt på LIA-trebanken, siden setningsavsluttende tegnsetting har et mellomrom foran seg.
Se annotasjonsretningslinjene  for LIA [her](https://tekstlab.uio.no/LIA/pdf/parseretningslinjer-lia12042019.pdf).



In [ ]:
from grewpy import Corpus, CorpusDraft

from ndt2ud.utils import set_spaceafter_from_text

# Last inn trebanken til et mutable CorpusDraft
draft = CorpusDraft(original_corpus)

draft.map(set_spaceafter_from_text, in_place=True)

# Skriv endringene tilbake til et immutable Corpus-objekt
corpus = Corpus(draft)

### Eksempel: endre upos

En python-funksjon som tar en graf som input kan spesifisere både søkefilteret og endringene som mappes til alle grafene i trebanken.

In [ ]:
from grewpy import CorpusDraft


def relabel_upos(graph, from_: str, to_: str):
    for node in graph:
        if "upos" in graph[node] and graph[node]["upos"] == from_:
            graph[node]["upos"] = to_
    return graph


draft = CorpusDraft(corpus)
draft.map(lambda x: relabel_upos(x, from_="anf", to_="PUNCT"), in_place=True)
modified_corpus = Corpus(draft)

## Kjør eksisterende omskrivingsregler på trebanken

In [ ]:
from grewpy import GRS

import ndt2ud

# Last inn reglene fra fil
src_root = Path(ndt2ud.__path__[0]).parent
NDT_to_UD_file = str(src_root / "rules" / "NDT_to_UD.grs")
NDT_to_UD_grs = GRS(NDT_to_UD_file)

# Bruk reglene for nynorsk på LIA : main_nn
corpus = NDT_to_UD_grs.apply(corpus, strat="main_nn")

## Skriv nye regler


### Steg 1: Søk etter setningsmønster i trebanken



Se retningslinjene for NDT-trebanken som definerer hvilke trekk og relasjoner vi evt. må endre. 

- Se Grew-dokumentasjonen for info om [`Request`](https://grew.fr/doc/request/) og syntaks for mønstrene (`pattern`-feltet i Request-objektet).

- Mønstersøk i eksisterende UD-versjon kan også gjøres i Grew Match: https://universal.grew.fr/?corpus=UD_Norwegian-Bokmaal@2.16

- Regelutikling kan også gjøres i Arborator, hvor vi har et privat prosjekt: [NDT_conversion_to_UD](https://arboratorgrew.elizia.net/#/projects/NDT_conversion_to_UD)


In [ ]:
from collections import namedtuple

from grewpy import Request


def create_node(token_features: dict):
    features = token_features.copy()
    del features["__RAW_MISC__"]
    del features["textform"]
    del features["wordform"]
    Node = namedtuple("Node", features.keys())
    node = Node(**features)
    return node


Edge = namedtuple("Edge", ["source", "label", "target"])


def view_search_results(request: Request, treebank: grewpy.Corpus):
    """Print the matching results in the treebank"""

    print(f"Antall treff: {treebank.count(request)}")
    print(request, "\n")

    print("Setninger som matcher mønsteret: ")
    for occ in treebank.search(request):  # type: ignore
        sent_id = occ["sent_id"]
        print(f"{sent_id=}")

        graph = treebank.get(sent_id)
        text = graph.to_sentence()
        print(f"{text=}")  # type: ignore

        for node_name, node_id in occ["matching"]["nodes"].items():
            token = create_node(graph.features[node_id])
            print(f"Node {node_name} ({node_id}): {token}")

        for edge_name, edge in occ["matching"]["edges"].items():
            e = Edge(**edge)
            print(f"Edge {edge_name} ({e.source} -> {e.target}): {e.label}")

        print()


### Eksempel: ukjent POS-tag "anf" 

Sjekk retningslinjene til [LIA](https://tekstlab.uio.no/LIA/pdf/parseretningslinjer-lia12042019.pdf) for info om LIA-spesifikke annotasjoner.

In [ ]:
from grewpy import CorpusDraft, Request

# Definer søkesmønsteret
request = Request().pattern("""
N [ upos=anf, form="«"|"»" ]; 
e: H -[IK]-> N; 
""")

view_search_results(request, corpus)

### Steg 2: Skriv en konverteringsregel

Skriv grewpy [Commands](https://grew.fr/doc/commands/) med Grew-syntaks som endrer setningsgrafen iht. UDs retningslinjer og pakk det inn i et GRS-objekt

In [ ]:
from grewpy.grs import (
    AddEdge,
    Command,
    Commands,
    DeleteEdge,
    DeleteFeature,
    GRSDraft,
    Package,
    RequestItem,
    Rule,
)

commands = Commands("""
   e.label = punct;
   N.upos = PUNCT;
   """)

# Alternatively:
# commands = Commands(
#     DeleteEdge("H", "IK", "N"),
#     AddEdge("H", "punct", "N"),
#     Command("N.upos = PUNCT")
# )

rule = Rule(request, commands)
grew_strategy = Package(dict(new_rule=rule))

grs_draft = GRSDraft(grew_strategy).onf()
grs = GRS(grs_draft)
corpus = grs.apply(corpus)

### Sammenlign trebanken før og etter endringene

In [ ]:
from pprint import pprint

original_matches = original_corpus.count(request)
request_matches = corpus.count(request)
command_matches = corpus.count(
    Request().pattern('N [ upos=PUNCT, form="«"|"»" ]; e: H -[punct]-> N')
)

print(f"Treff på regelmønster FØR endringen: {original_matches}")  # Burde være 0 nå
print(f"Treff på regelmønster ETTER endringen: {request_matches}")  # Burde være 0 nå
print(f"Treff på nytt mønster etter endringen: {command_matches}")

In [ ]:
# Se over statistikk for trebanken
n_sentences = corpus.count(Request())  # Empty request will match all sentences
print(f"Antall setninger i trebanken: {n_sentences}")

feature_stats = corpus.count_feature_values(
    exclude=["form", "lemma", "textform", "wordform"]
)
pprint(feature_stats)

### Visualiser setningsgrafer

In [ ]:
# Check if a norwegian spacy model exists in the virtual environment
try:
    import nb_core_news_md
except ModuleNotFoundError:
    print("Download a Norwegian spacy model to visualise graphs with Displacy")
    !python -m spacy download nb_core_news_md

In [ ]:
from grewpy.network import GrewError
from IPython.display import HTML, SVG, display
from spacy import displacy
from spacy_conll import init_parser
from spacy_conll.parser import ConllParser

from ndt2ud.visualize import visualize_graph_dot


def visualize_graph(graph):
    sent_id = graph.meta["sent_id"]
    print(f"Sentence ID: {sent_id}")
    print(f"Text: {graph.to_sentence()}")
    output_name = f"graph_{sent_id}"
    try:
        # graph.to_svg() has a known issue with some grewpy installations, but may work in some environments
        svg_result = graph.to_svg()
        Path(f"{output_name}.svg").open("w", encoding="utf-8").write(svg_result)
        visual_graph = SVG(svg_result)
    except GrewError as e:
        conllstr = graph.to_conll()

        if "nlp" not in locals():
            nlp = ConllParser(init_parser("nb_core_news_md", "spacy"))
        try:
            doc = nlp.parse_conll_text_as_spacy(conllstr)  # type:ignore
            visual_graph = HTML(
                displacy.render(
                    doc,
                    style="dep",
                    jupyter=False,
                    options={"compact": False, "color": "green", "distance": 100.0},
                )
            )

        except ValueError:
            print(f"Using alternative visualization method with dot and graphviz")
            result = visualize_graph_dot(graph, output_name)
            visual_graph = SVG(filename=result)

    display(visual_graph)

### Sammenlign grafene før og etter regelen

In [ ]:
# Hent ut en bestemt setningsgraf med setnings-ID
sent_id = "76"

visualize_graph(original_corpus[sent_id])

In [ ]:
# Se på samme grafen etter omskrivingsregelen
visualize_graph(corpus[sent_id])

## Lagre regler i filer 

Lagre reglene i GREW-format i en `grs`-fil, med valgt strategi. 


In [ ]:
grs_draft.save("new_LIA_rule.grs")